In [ ]:
# Automatic Dataset Download for Augmentation Tutorial
import os
import requests
from pathlib import Path
from tqdm import tqdm

def download_augmentation_samples():
    """Download sample images for augmentation demonstrations"""
    data_dir = Path("../data")
    data_dir.mkdir(parents=True, exist_ok=True)
    
    # High-quality histopathology samples for augmentation
    samples = {
        'histology_normal.jpg': 'https://github.com/jamesdolezal/slideflow/raw/master/docs/source/_static/img/tile_3.jpg',
        'histology_tumor.jpg': 'https://github.com/jamesdolezal/slideflow/raw/master/docs/source/_static/img/tile_4.jpg',
        'tissue_sample.png': 'https://raw.githubusercontent.com/PathologyDataScience/TCGA-BRCA-WSI-Tiles/main/sample_tiles/normal_001.png'
    }
    
    print("🔄 Preparing augmentation sample data...")
    downloaded_count = 0
    
    for filename, url in samples.items():
        filepath = data_dir / filename
        if not filepath.exists():
            try:
                print(f"📥 Downloading {filename}...")
                response = requests.get(url, stream=True)
                response.raise_for_status()
                
                total_size = int(response.headers.get('content-length', 0))
                with open(filepath, 'wb') as f, tqdm(
                    desc=filename,
                    total=total_size,
                    unit='iB',
                    unit_scale=True,
                    unit_divisor=1024,
                ) as pbar:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            size = f.write(chunk)
                            pbar.update(size)
                
                print(f"✅ Successfully downloaded {filename}")
                downloaded_count += 1
            except requests.RequestException as e:
                print(f"❌ Failed to download {filename}: {e}")
                # Create placeholder if download fails
                try:
                    import numpy as np
                    from PIL import Image
                    placeholder = np.random.randint(0, 255, (256, 256, 3), dtype=np.uint8)
                    Image.fromarray(placeholder).save(filepath)
                    print(f"📝 Created placeholder for {filename}")
                except Exception as pe:
                    print(f"❌ Could not create placeholder: {pe}")
        else:
            print(f"✅ {filename} already available")
    
    print(f"🎯 Augmentation data ready! {downloaded_count} new downloads completed")
    return data_dir

# Setup augmentation sample data
augmentation_data_dir = download_augmentation_samples()
print(f"📁 Augmentation samples located at: {augmentation_data_dir.absolute()}")

## Basic Image Augmentation

Data augmentation is a technique to increase the diversity of your training set by applying random transformations to your images. This helps prevent overfitting.

**Our Goals:**
1.  Apply random horizontal flips.
2.  Apply random rotations.
3.  Combine multiple augmentations.

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import random

# Load our sample tile
tile_path = '../data/sample_tile.png'
original_image = Image.open(tile_path)

### 1. Horizontal Flip

A horizontal flip is a common and effective augmentation.

In [ ]:
flipped_image = original_image.transpose(Image.FLIP_LEFT_RIGHT)

fig, axes = plt.subplots(1, 2, figsize=(8, 4))
axes[0].imshow(original_image)
axes[0].set_title('Original')
axes[1].imshow(flipped_image)
axes[1].set_title('Flipped')
plt.show()

### 2. Rotation

We can also rotate the image by a random angle.

In [ ]:
rotated_image = original_image.rotate(45)

fig, axes = plt.subplots(1, 2, figsize=(8, 4))
axes[0].imshow(original_image)
axes[0].set_title('Original')
axes[1].imshow(rotated_image)
axes[1].set_title('Rotated 45 degrees')
plt.show()

## ✅ Final Check

Let's create a simple augmentation function and test it.

In [ ]:
def augment_image(image):
    """Applies a random flip and rotation."""
    if random.random() > 0.5:
        image = image.transpose(Image.FLIP_LEFT_RIGHT)
    
    angle = random.choice([0, 90, 180, 270])
    image = image.rotate(angle)
    return image

augmented_image = augment_image(original_image)

# This check is visual, as the output is random.
# In a real test, you might check that the output image
# has the same size as the input.
assert original_image.size == augmented_image.size

print("SUCCESS: Augmentation function created and tested.")
plt.imshow(augmented_image)
plt.title('Randomly Augmented')
plt.show()